# Boosting Models
This notebook follows the orders

## Loading our data...

In [44]:
import numpy as np
import pandas as pd
from utils import read_data, process_time, merge_data
import seaborn as sns
import matplotlib.pyplot as plt
import sys

sys.path.append("../../main/datasets/")

In [45]:
!ls  ../../main/datasets/

1.0v.zip


In [46]:
infos, items, orders = read_data("../../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


In [111]:
orders.head()

,time,transactID,itemID,order,salesPrice,days,days_backwards,group_backwards
0,2018-01-01 00:01:56,2278968,450,1,17.42,1,180,13
1,2018-01-01 00:01:56,2278968,83,1,5.19,1,180,13
2,2018-01-01 00:07:11,2255797,7851,2,20.47,1,180,13
3,2018-01-01 00:09:24,2278968,450,1,17.42,1,180,13
4,2018-01-01 00:09:24,2278968,83,1,5.19,1,180,13


In [ ]:
# def promo_inference(orders, mode='mode'):
pricesMode = orders.groupby('itemID').agg(salesPriceMode=('salesPrice', lambda x:x.value_counts().index[0]))
orders[['itemID', 'salesPrice']].merge(orders, right_on='itemID', left_on='itemID')

## Preprocessing our orders

These cells were taken from Bruno's "1.1-First Model" Notebook, that can be found in this repository.

In [47]:
process_time(orders)

In [59]:
aggs = {"order": "sum", "salesPrice": "mean"}
df = orders.groupby(["group_backwards", "itemID"],
                    as_index=False).agg(aggs)
df.rename(columns={x: x+"_"+y for x, y in aggs.items()}, inplace=True)

In [60]:
df.head()

,group_backwards,itemID,order_sum,salesPrice_mean
0,1,1,3,3.43
1,1,3,140,14.04
2,1,4,145,14.10
3,1,5,1,7.48
4,1,7,1,34.39


In [62]:
merge_data(df, items, infos)

,group_backwards,itemID,order_sum,salesPrice_mean,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice,simulationPrice,promotion
0,1,1,3,3.43,0,1,4.38,1,1,1,8.84,3.43,NaN
1,3,1,31,3.11,0,1,4.38,1,1,1,8.84,3.43,NaN
2,4,1,3,3.11,0,1,4.38,1,1,1,8.84,3.43,NaN
3,5,1,299,3.11,0,1,4.38,1,1,1,8.84,3.43,NaN
4,6,1,2,3.11,0,1,4.38,1,1,1,8.84,3.43,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39510,13,9024,5,6671.46,186,233,0.00,8,44,8,3308.60,6671.46,NaN
39511,13,9319,30,362.79,227,248,5.00,8,44,8,218.01,362.79,NaN
39512,13,9327,19,36.50,227,248,0.00,8,44,8,101.40,36.50,2018-07-04
39513,13,9603,6,411.60,186,233,0.00,8,44,8,840.00,411.60,NaN
